In [1]:
import os
import mne
from mne_bids import BIDSPath, read_raw_bids

In [2]:
# Путь к корневой папке датасета (ds004504):
BIDS_ROOT = "/path/to/ds004504"

In [3]:
# Пример: список испытуемых.
# Можно найти их автоматически, посмотрев подпапки sub-xx, либо прописать вручную.
SUBJECTS = ["001", "002"]  # в реальном датасете ds004504 другие индексы

In [4]:
# Параметры фильтрации:
L_FREQ = 1.0   # нижняя частота
H_FREQ = 40.0  # верхняя частота

In [5]:
# Папка, куда будем сохранять отфильтрованные файлы
OUT_DIR = os.path.join(BIDS_ROOT, "derivatives", "filtered_eeg")
os.makedirs(OUT_DIR, exist_ok=True)

OSError: [Errno 30] Read-only file system: '/path'

In [ ]:
def main():
    for subj in SUBJECTS:
        # Создаём BIDSPath
        # Смотрите, какие именно "task" и "session" есть в вашем датасете ds004504
        # Часто resting-state называют "task-rest" или "task-eyesopen"
        bids_path = BIDSPath(
            subject=subj,
            task="rest",       # или другое название из файла, проверьте
            suffix="eeg",      # вид файла (eeg)
            extension=".edf",  # расширение
            root=BIDS_ROOT
        )

        # Читаем raw-данные
        print(f"Reading BIDS data: {bids_path}")
        try:
            raw = read_raw_bids(bids_path=bids_path, verbose=False)
        except FileNotFoundError:
            print(f"File for subject={subj} not found. Skipping...")
            continue

        # Выгружаем сигналы в память
        raw.load_data()

        # Фильтрация:
        print(f"Filtering data: {L_FREQ}–{H_FREQ} Hz")
        raw.filter(l_freq=L_FREQ, h_freq=H_FREQ)

        # Устанавливаем общее среднее (average) реферирование (если это нужно)
        # Другие варианты: raw.set_eeg_reference('TP9', 'TP10') для мочек ушей, и т.д.
        raw.set_eeg_reference(ref_channels='average', projection=False)

        # (Опционально) Дополнительно можно убрать первые X секунд, если там шум
        # raw.crop(tmin=5.0)  # например, убрать 5 секунд в начале

        # (Опционально) выполнить ICA/ASR и удалить артефакты
        # ...

        # Сохраняем результат в папку derivatives/filtered_eeg
        out_fname = os.path.join(OUT_DIR, f"sub-{subj}_task-rest_filtered-raw.fif")
        print(f"Saving filtered data to: {out_fname}")
        raw.save(out_fname, overwrite=True)

    print("Done filtering!")

if __name__ == "__main__":
    main()